# Practical session 2: Implementing reactive behaviors

## Preliminary notes

**Reminders :** 
- Save your work as indicated at the beginning of the `session_1.ipynb` notebook.
- Each time you encounter a cell containing code in this notebook (as in the cell starting with `from vivarium.controllers ...` below), you have to execute it by clicking on the cell and pressing `Shift+Enter` (unless it is explicitly specified not to do it).

<!-- TODO: load scene !-->

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Start the simulator:

In [2]:
controller.run(threaded=True)

If at one point things are not going as expected (e.g. the agent doesn't move the way it should in the simulator), do the following steps:
- Stop any code that can still be executing by pressing the "stop-like" button in the top menu bar of this document. 
- Stop the simulator by executing (no need to do it now though):

In [3]:
controller.stop()

- Restart the notebook by clicking `Kernel -> Restart` in the menu.
- Re-open the session by executing (don't re-open it now if you haven't closed it by executing the previous cell):

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()
controller.run(threaded=True)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


0.7052901983261108 0.7022781372070312
0.7052901983261108 0.7022781372070312
0.7054997682571411 0.7024980783462524
0.7060219049453735 0.7030462026596069
0.7066349983215332 0.7036898136138916
0.7072725296020508 0.7043592929840088
0.7079157829284668 0.7050348520278931
0.7085594534873962 0.7057108879089355
0.7092018723487854 0.7063858509063721
0.7098426818847656 0.7070590853691101
0.7104817032814026 0.7077305316925049
0.7111188769340515 0.7084000110626221
0.7117542028427124 0.709067702293396
0.7123878002166748 0.709733247756958
0.7130193710327148 0.7103968262672424
0.7136491537094116 0.7110583782196045
0.7142770290374756 0.7117179036140442
0.714902937412262 0.7123753428459167
0.715526819229126 0.7130305767059326
0.716148853302002 0.7136836051940918
0.716768741607666 0.7143344879150391
0.7173867225646973 0.7149831056594849
0.7180026173591614 0.7156294584274292
0.7186164259910583 0.7162735462188721
0.7192281484603882 0.7169153094291687
0.7198377847671509 0.7175547480583191
0.7204452753067017

We can create an alias for the agent again since there is only one:

In [2]:
agent = controller.agents[0]

## Behaviors

In the last practical session we saw how to set the agent left and right wheel speeds as well as how to read the values returned by the left and right proximeters. We programmed a first simple behavior where the agent slows down when approaching an obstacle. 
Here is a possible solution for this behavior:

In [6]:
# Repeat 200 times the indented code:
for i in range(200):
    # Read the proximeter values and store them in the "left" and "right" variables
    left, right = agent.sensors()
    
    # Compute the sum of the values returned by the left and right proximeters.
    # This sum will be between 0 and 2 because both "left" and "right" are between 0 and 1
    sum_of_proxs = left + right
    
    # Compute the activation that will be applied to both wheels. 
    # The closer the obstacle (i.e. the higher the value returned by the proximeters), the lower the wheel activation should be.
    # Note that wheel activation is bounded between 0 and 1
    wheel_activation = 1.0 - sum_of_proxs / 2.0
    
    # Set the activation of both wheels to the value we just have computed
    agent.right_motor = wheel_activation
    agent.left_motor = wheel_activation
    
    # Waits for 100 milliseconds before starting the next iteration (to avoid overloading you computer)
    controller.wait(0.1)

agent.right_motor = agent.left_motor = 0.0

## Practical definition of a behavior

The example behavior defined above illustrates the general structure of a behavior. 

**Definition:** a behavior consists of a loop repeated at a certain frequency where (1) the values of relevant sensors are read, (2) some computation is performed using these values and (3) commands are sent to the agent motors according to the result of this computation.

In the example above, the frequency of the behavior is approximately 10 Hz, i.e. the core of the loop is executed approximately 10 times per second (because we wait for 0.1s at the end of each iteration). This is an approximation because we don't take into account the time needed to execute the instructions occurring before the waiting period. Step (1) corresponds to the reading of the left and right proximeters activations. Step (2) corresponds to the computation of `wheel_activation` according to the sum of the proximeter activations. Finally, Step (3) corresponds to setting the speed of both motors to the value of `wheel_activation`.

Note that the code above will take a while to be executed (approximately `1000 * 0.1 = 100` seconds, since the loop is repeated 1000 times). During this time, you can't execute anything else in this notebook. To stop the execution before it terminates by itself, you have to press the "stop-like" button in the top menu bar of this document. 

This approach has three major drawbacks:
- Only one behavior can run at a time.
- The behavior has a fixed duration (at one point it will stop)
- We can't stop a behavior programmatically (instead we have to press the "stop-like" button).

To overcome these problems, we provide a more flexible method for defining and executing behaviors. Let's rewrite the behavior above using that method. First make sure the previous code is not still being executed by pressing the "stop-like" button in the top menu bar of this document. Now, defining a behavior boils down to defining a function which includes the core of the behavioral loop:

In [7]:
# The code in this cell defines a function called slow_down (first line),
# which takes as argument the agent (first line, in parenthesis),
# and returns the left and right speed activation to be applied to the motors (last line)

def slow_down(agent):
    # Step (1): read the sensor values
    left, right = agent.sensors()
    
    # Step (2): do some computation
    sum_of_proxs = left + right
    wheel_activation = 1.0 - sum_of_proxs / 2.0
    
    # Step (3): return the motor activations
    return wheel_activation, wheel_activation

The cell above defines a function called `slow_down`. In computer programming, a function is a sequence of instructions that perform a specific task depending on some parameters (called the arguments of the function) and that returns a result. In this sense it is very similar to the mathematical definition of a function, as for example when we write `y = f(x)`, where `f` is the name of the function, `x` is its argument, and `y` is the result.

As seen above, the definition of a function in Python starts with the keyword `def`, followed by the arbitrary name we choose for the function (here we called it `slow_down` to reflect the purpose of the behavior defined in it). Then come the arguments of the function in parenthesis (in our case it will be the variable representing the agent, called `agent`) and finally the symbol `:`. Below the first line, you find the instructions that this function will execute when it will be called. Those instructions need to be intended right with respect to the first line. In this example, the instructions are the exact same as in the core of the previous `for` loop, except that:
- we omit the last line `controller.wait(0.1)` (the frequency at which the behavior will be executed will be set in more rigorous way below),
- we don't directly set the motor activations using `agent.left_motor` and `agent.right_motor`. Instead, we *return* the values of the motor activations in the last line and they will be automatically sent to the agent motors when the behavior will be executed. In the last line, the values after the `return` keyword have to be the left and right speed activation (in this order). Both activations have to be between 0 and 1. (In the `slow_down` behavior above, both activations are the same since we don't want the agent to turn).

Note that a function definition, as the one above, does not execute the instructions contained in it, it only defines them so that they can be executed later when the function will be *called*. In our case, we will not explicitly call the function, instead it will be done behind the scene when we will start the behavior on the agent (see below).

Once the behavior is defined as a function, we can attach it to the agent by executing:


In [8]:
agent.attach_behavior(slow_down)

The line above means: attach the behavior defined in the function `slow_down` to the `agent` and set the frequency at which it will be repeated to ~10Hz. Note that this instruction does not execute the behavior on the robot, it only informs the robot that the behavior is available to it.

In order to actually start the behavior, we have to execute:

In [9]:
agent.check_behaviors()

Available behaviors: ['slow_down']
No active behaviors


In [10]:
agent.start_behavior("slow_down")

You should now see the agent executing the exact same behavior as before (if the agent is already close to an obstacle, move it to a more open space to observe it slowing down).

The line above means: start running the previously attached `slow_down` behavior on the `agent`. Executing the above line will basically do the same thing as executing the `for` loop at the start of this document. Here, the function `slow_down` will be executed at a frequency of 10Hz as set in the previous cell (i.e. 10 times per second), indefinitely. 

Using this method has the following advantages over the previous method using the `for` loop:
- It is more compact to write and it will allow to better structure your code when you will have to deal with multiple behaviors and multiple agents.
- It better manages the frequency at which the behavior is run. Now the behavior runs at exactly 10Hz, whereas in the previous method we could only approximate the true frequency.
- It is not blocking as the previous method was. This means that you can still use this notebook while the behavior is running on the agent. For example, let's read the proximeter activations while the agent is still executing the `slow_down` behavior:

In [13]:
agent.sensors()

[0.0, 0.7907664775848389]

Each time you execute the cell above, you should see the proximeter activation changing because the agent is moving. However, you should avoid setting motor values while a behavior is running since this could conflict with the behavior also setting those values. When a behavior is started, it runs indefinitely until you explicitly tell it to stop. To do so, you have to execute:

In [14]:
agent.stop_behavior("slow_down")

Note that the agent will continue moving using the last wheel speeds that were set by the behavior. You can set both wheel speeds to 0 by executing:

In [15]:
agent.stop_motors()

Finally, if you don't want the behavior to be attached to the agent anymore (for example if you want to test another behavior), you can execute:

In [16]:
agent.detach_behavior("slow_down")

At anytime, you can check what behaviors are attached to the agent with:

In [17]:
agent.check_behaviors()

No behaviors attached


**Q1:** To make sure you correctly understand how to attach, start, stop and detach a behavior, complete the following code:

In [18]:
# TODO : decide a specific print for the check behavior method 

# First we make sure that no behavior is attach to the agent:
agent.detach_all_behaviors()

# When checking, it should print "No behavior attached":
agent.check_behaviors() # This will print "No behavior attached"


# Write just below this line the code that attaches the slow_down behavior
agent.attach_behavior(slow_down)

agent.check_behaviors() # This will print "Behavior "slow_down" is attached and NOT STARTED"


# Write just below this line the code that starts the slow_down behavior
agent.start_behavior("slow_down")

agent.check_behaviors() # This will print "Behavior "slow_down" is attached and STARTED"


# Write just below this line the code that stops the slow_down behavior
agent.stop_behavior("slow_down")

agent.check_behaviors() # This will print "Behavior "slow_down" is attached and NOT STARTED"


# Write just below this line the code that detaches the slow_down behavior
agent.detach_behavior("slow_down")

agent.check_behaviors() # This will print "No behavior attached"

No behaviors attached
Available behaviors: ['slow_down']
No active behaviors
Available behaviors: ['slow_down']
active behaviors: ['slow_down']
Available behaviors: ['slow_down']
No active behaviors
No behaviors attached


Let's summarize the method we have just describe to define, attach, start, stop and detach a behavior:

In [19]:
# First, detach all the behaviors that might still be attached to the agent
# (it is a good practice to do it each time you want to define a new behavior, or modify an existing one):
agent.detach_all_behaviors()

In [20]:
# Define a behavior where the agent progressively slows down when it approaches an obstacle:
def slow_down(agent):
    # Step (1): read the sensor values
    left, right = agent.sensors()
    
    # Step (2): do some computation
    sum_of_proxs = left + right
    wheel_activation = (2 - sum_of_proxs) / 2
    
    # Step (3): return the motor activations
    return wheel_activation, wheel_activation

In [21]:
# Attach this behavior to the agent, specifying the frequency (in Hz) at which it will be executed
agent.attach_behavior(slow_down)
# Start the behavior in the simulator
agent.start_behavior(slow_down)

When executing the code above, you should see the behavior being executed on the agent in the simulator. Then, to stop and detach the behavior:

In [22]:
agent.stop_behavior(slow_down)
agent.detach_behavior(slow_down)

An alternative way is to stop and detach all the behaviors running on the agent. This avoids having to specify the name of the behavior (`slown_down` in the cell above) and also stops systematically the behavior before detaching it:

In [23]:
agent.detach_all_behaviors()

You might also want to stop the agent wheels (i.e. setting both wheel activations to 0) by executing:

In [24]:
agent.stop_motors()

## Implementing the Braitenberg vehicles

Let's now practice a bit. Remember the Braitenberg Vehicle examples we have seen in [this slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0). Those vehicles are very similar to the agents in the simulator. 
- It is equipped with two sensors that are activated according to the proximity of a source. With the agent, each proximeter sensor returns a value between 0 and 1 that is inversely proportional to the distance from the closest obstacle it perceives (the closer the obstacle, the highest to proximeter activation).
- It is equipped with two motors allowing the agent to move. With the agent, we can set the activation of each wheel independently with a value between 0 and 1 (where 1 means maximum speed). 
- A behavior links sensor activations to motor activations. In the Braitenberg vehicles, this is achieved through connections that are either excitatory (the activity of the sensor increases the activity of the motor it is connected to) or inhibitory (the activity of the sensor decreases the activity of the motor it is connected to). In the agent, we have seen above that we can define a behavior as a function that (1) read the sensor activities (2) perform some computation and (3) use the result of that computation to set the wheel speed. 

Therefore, we can implement in the agent the various types of vehicle behaviors shown in the slide, where defining excitatory and inhibitory connections will be done through Step (2) above (*perform some computation*). We have actually already done it with the `slow_down` behavior we have defined above.  

**Q2:** Define verbally the `slow_down` behavior in term of inhibitory and excitatory connections (do it by double clicking on the next cell). Your answer must look like this (where `TO_FILL` is either the word "excitatory" or "inhibitory"):
- The activity of the left sensor is connected to the left wheel through a TO_FILL connection.
- The activity of the left sensor is connected to the right wheel through a TO_FILL connection.
- The activity of the right sensor is connected to the left wheel through a TO_FILL connection.
- The activity of the right sensor is connected to the right wheel through a TO_FILL connection.

*Double click on this cell and replace this text by your answer*

Let's see how to define the `fear` behavior illustrated in [the slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0) using the method we have seen: 

In [3]:
def fear(agent):
    left, right = agent.sensors()
    return left, right

That's pretty easy, isn't it? As illustrated in the slide, the `fear` behavior simply consists in the left sensor exciting the left wheel, and the right sensor exciting the right wheel. Therefore, the simplest way of programming this behavior is to directly map the left and right sensor activations to the left and right wheel speed, respectively. This is what is done in the function definition just above.

Let's now analyze the properties of this `fear` behavior in more detail. Attach and start the `fear` behavior by executing the cell below, and observe how the agent behaves.

In [11]:
agent.detach_all_behaviors()  # Just in case a behavior is still attached

agent.attach_behavior(fear)
agent.start_all_behaviors()

In [5]:
agent.sensors()

[0.0, 0.0]

In [8]:
agent.detach_all_behaviors()

In [10]:
agent.left_motor = agent.right_motor = 0.

In [6]:
agent.infos()

Agent Overview:
--------------------
Entity Type: AGENT
Position: x=199.09, y=6.01
Behaviors:
  - fear: Function=fear, Weight=1.0
Sensors: Left=0.00, Right=0.00
Motors: Left=0.00, Right=0.00


**Q3:** Use the cell below to answer the following questions.
<!-- TODO: change that because there is no walls/corners !-->
1. What happens when the activity of both sensors is null? (i.e. no obstacle is detected.) Why?
2. How does the agent react when it detects an obstacle? (e.g. an object.) Why?
3. How does the agent reacts when it approaches a corner?  Why?
4. How does the agent reacts when it approaches perpendicularly to a wall?  Why?
5. Imagine a small animal equipped with such a behavior in the wild. What would be its evolutionary advantages and drawbacks? (could it escape from a predator? could it collect food? Could it hide itself?)

*Double click on this cell and replace this text by your answer*

Before programming the next behavior, let's first reduce the maximum speed of the agent (not implemented):

In [ ]:
# agent.max_speed = 5

**Q4:** Program the `aggression` behavior illustrated in [the slide](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0), which consists of crossed excitatory connections. 

In [3]:
def aggression(agent):
    left, right = agent.sensors()
    print(left, right)
    return right, left

Before executing the behavior you have defined in the cell just above, first detach the previous one and immobilize the agent:

In [4]:
agent.detach_all_behaviors()
agent.stop_motors()

Then attach the `aggression` behavior and start it:

In [5]:
agent.attach_behavior(aggression)
agent.start_all_behaviors()

**Q5:** Use the cell below to answer the following questions.
3. How does the agent reacts when it approaches a wall?  Why?
2. How does the agent react when close to a moveable object (the kind of trash bins in the scene) Why?
4. Imagine an animal equipped with such a behavior in the wild. What would be its evolutionary advantages and drawbacks? (could it escape from a predator? could it catch preys? Could it hide itself? Could it move things?)

*Double click on this cell and replace this text by your answer*

That's it for this practical session. You can now close the session:

In [7]:
controller.stop()